In [1]:
import pandas as pd
import geopandas as gpd
import pickle
from util import get_service_area, load_Baltimore_blocks
OD = pd.read_csv('../../raw_data/md_od_main_JT00_2020.csv.gz')
stations = gpd.read_file('../../raw_data/Baltimore Red Line- Stations.csv')
segments = gpd.read_file('../../raw_data/Baltimore Red Line- Alignment.csv')

gdf_Baltimore, md_rac_df, md_wac_df = load_Baltimore_blocks()

service_area = get_service_area(gdf_Baltimore, stations)

In [2]:
residents = md_rac_df[md_rac_df['h_geocode'].astype(str).str.startswith('24510')]
residents = residents[['h_geocode', 'CE01', 'CE02', 'C000']]
residents['density'] = (residents['CE01'] + residents['CE02'])/residents['C000']

In [3]:
gdf = service_area.merge(residents, left_on='GEOID20', right_on='h_geocode')

In [4]:
from util import get_Baltimore_city_map, save_png
from folium import Choropleth
city_map =  get_Baltimore_city_map(segments=segments, zoom_start=13)

geojson_data = gdf.to_json()
legend_name = 'Density of Mid-to-lower-income Workers'

# Create a choropleth map to visualize the 'mid_to_lower' column
Choropleth(
    geo_data=geojson_data,
    data=gdf,
    columns=['GEOID20', 'density'],
    key_on='feature.properties.GEOID20',
    line_color='white',
    line_weight=0.1,
    fill_color='Blues',
    fill_opacity=0.7,
    legend_name=legend_name,
).add_to(city_map)
save_png(city_map, 'figures/density_of_mid_to_lower_income_workers')
city_map

In [5]:
gdf.explore(column='density', cmap='Blues')

In [17]:
#gdf_Baltimore.geometry = gdf_Baltimore.geometry.centroid
gdf_Baltimore[gdf_Baltimore['GEOID20'] == '245102605013028']

,geometry,GEOID20
83486,POINT (-76.54498 39.28599),245102605013028
